In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
datadf = pd.read_csv("../data/the_rise_of_healthcare_jobs_disclosed_data_by_msa.csv")
datadf.head(10)

In [ ]:
datadf["metro13"].unique()

In [ ]:
import json

import altair as alt
import pandas as pd

# import natsort
alt.data_transformers.enable("vegafusion")

In [ ]:
jsonFile = json.load(Path.open("../data/2021_US_CBSA.geojson"))
cbsa = pd.DataFrame(jsonFile["features"])
cbsa["Location Name"] = cbsa["properties"].apply(lambda x: x["NAME"])
cbsa["Location ID"] = cbsa["properties"].apply(lambda x: x["CBSAFP"])
cbsa = cbsa[~cbsa["Location Name"].str.contains("HI|AK|PR", regex=True)]

In [ ]:
jsonFile = json.load(Path.open("../data/2021_US_States.geojson"))
states = pd.DataFrame(jsonFile["features"])
states["State Name"] = states["properties"].apply(lambda x: x["NAME"])
states["State Abbreviation"] = states["properties"].apply(lambda x: x["STUSPS"])
states = states[
    ~states["State Abbreviation"].isin(["HI", "AK", "PR", "VI", "GU", "MP", "AS"])
]

In [ ]:
# Choropleth Base Map
alt.Chart(cbsa, title="CBSA").mark_geoshape(stroke="black", strokeWidth=0.75).encode(
    color=alt.value("lightgrey"),
    tooltip=[
        alt.Tooltip("Location Name:N", title="Location Name:"),
        alt.Tooltip("Location ID:N", title="Location ID:"),
    ],
).project(type="albers").properties(width=1000, height=800)

In [ ]:
# Choropleth Base Map
alt.Chart(states, title="U.S. States").mark_geoshape(
    stroke="black", strokeWidth=0.75
).encode(
    color=alt.value("lightgrey"),
    tooltip=[alt.Tooltip("State Name:N", title="State Name:")],
).project(type="albers").properties(width=1000, height=800)

In [ ]:
# Reshape df to long format
value_cols = [c for c in datadf.columns if c not in ["metro13", "metro_title"]]
df_long = datadf.melt(
    id_vars=["metro13", "metro_title"],
    value_vars=value_cols,
    var_name="indicator",
    value_name="value",
)

In [ ]:
indicator_dropdown = alt.binding_select(options=value_cols, name="Select variable: ")
indicator_selection = alt.selection_point(fields=["indicator"], bind=indicator_dropdown)
point_selection = alt.selection_point(fields=["metro_title"])

base = (
    alt.Chart(states)
    .mark_geoshape(fill="lightgray", stroke="black", strokeWidth=0.5)
    .encode(tooltip=alt.Tooltip("State Name:N", title="State:"))
)

cbsa_layer = (
    alt.Chart(df_long)
    .mark_geoshape(stroke="#888888", strokeWidth=0.8)
    .encode(
        color=alt.Color(
            "value:Q",
            scale=alt.Scale(scheme="viridis"),
            legend=alt.Legend(orient="bottom"),
        ),
        opacity=alt.condition(point_selection, alt.value(1.0), alt.value(0.25)),
        tooltip=[
            alt.Tooltip("metro_title:N", title="Metropolitan Area"),
            alt.Tooltip("indicator:N", title="Variable"),
            alt.Tooltip("value:Q", title="Value"),
        ],
    )
    .transform_lookup(
        lookup="metro13",
        from_=alt.LookupData(
            cbsa, key="Location ID", fields=["geometry", "type", "Location Name"]
        ),
    )
    # .add_params(indicator_selection)
    .transform_filter(indicator_selection)
    # .project(type="albers")
    # .properties(width=900, height=600, title="CBSA-Level Data by Indicator")
)

state_layer = (
    alt.Chart(df_long)
    .mark_geoshape(stroke="#414141", strokeWidth=0.6)
    .encode(
        color=alt.Color(
            "value:Q",
            scale=alt.Scale(scheme="viridis"),
            legend=alt.Legend(orient="bottom"),
        ),
        opacity=alt.condition(point_selection, alt.value(1.0), alt.value(0.25)),
        tooltip=[
            alt.Tooltip("metro_title:N", title="State"),
            alt.Tooltip("indicator:N", title="Variable"),
            alt.Tooltip("value:Q", title="Value"),
        ],
    )
    .transform_lookup(
        lookup="metro_title",
        from_=alt.LookupData(
            states,
            key="State Abbreviation",
            fields=["geometry", "type", "Location Name"],
        ),
    )
    # .add_params(indicator_selection)
    .transform_filter(indicator_selection)
    # .project(type="albers")
    # .properties(width=900, height=600, title="State-Level Data by Indicator")
)

layered_map = (base + state_layer + cbsa_layer).add_params(
    indicator_selection, point_selection
)
layered_map = (
    layered_map.project(type="albers")
    .properties(width=1000, height=600, title="CBSA and State-Level Data by Indicator")
    .interactive()
)

# layered_map

In [ ]:
bar_chart = (
    alt.Chart(df_long, title="Values by Region")
    .mark_bar()
    .encode(
        x=alt.X(
            "metro_title:N",
            sort=alt.SortField(
                field="value",  # dynamically sort by value
                order="ascending",  # or 'descending' if you prefer
            ),
            title=None,
        ),
        y=alt.Y("value:Q", title="Value"),
        color=alt.condition(
            point_selection,
            alt.Color("value:Q", scale=alt.Scale(scheme="viridis")),
            alt.value("lightgray"),
        ),
        opacity=alt.condition(point_selection, alt.value(1.0), alt.value(0.6)),
        tooltip=[
            alt.Tooltip("metro_title:N", title="Region"),
            alt.Tooltip("value:Q", title="Value"),
        ],
    )
    .add_params(point_selection)
    .transform_filter(indicator_selection)
    .properties(width=1200, height=200)
)

layered_map & bar_chart

# Below is code for shapefile generation

In [ ]:
import io
import zipfile

import requests

url = "https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_cbsa_5m.zip"

r = requests.get(url, timeout=10)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("../data/cb_2021_us_cbsa_5m")
z.close()

url2 = "https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_state_5m.zip"

r2 = requests.get(url2, timeout=10)
z2 = zipfile.ZipFile(io.BytesIO(r2.content))
z2.extractall("../data/cb_2021_us_state_5m")
z2.close()

In [ ]:
# Convert extracted shapefiles to GeoJSON
import geopandas as gpd

shp_dirs = {
    "cbsa": Path("../data/cb_2021_us_cbsa_5m"),
    "states": Path("../data/cb_2021_us_state_5m"),
}

out_paths = {
    "cbsa": Path("../data/2021_US_CBSA_auto.geojson"),
    "states": Path("../data/2021_US_States_auto.geojson"),
}

epsg_wgs84 = 4326

for key, d in shp_dirs.items():
    shp_files = sorted(d.glob("*.shp"))
    if not shp_files:
        print(f"No .shp found in {d!s}; skipping {key}")
        continue

    shp_path = shp_files[0]
    gdf = gpd.read_file(shp_path)

    # ensure WGS84 (GeoJSON-friendly)
    if gdf.crs is None or gdf.crs.to_epsg() != epsg_wgs84:
        gdf = gdf.to_crs(epsg=epsg_wgs84)

    # keep common identifier/name columns where available
    if key == "cbsa":
        id_cols = [c for c in ("CBSAFP", "GEOID", "NAME") if c in gdf.columns]
    else:
        id_cols = [
            c for c in ("STATEFP", "STUSPS", "GEOID", "NAME") if c in gdf.columns
        ]

    # cast identifier columns to string to avoid numeric/float issues in GeoJSON props
    for c in id_cols:
        gdf[c] = gdf[c].astype(str)

    # write GeoJSON
    out = out_paths[key]
    gdf.to_file(out, driver="GeoJSON")
    print(f"Wrote {len(gdf)} features to {out}")

Wrote 939 features to ../data/2021_US_CBSA_auto.geojson
Wrote 56 features to ../data/2021_US_States_auto.geojson


In [87]:
# --- Load the original GeoJSONs
gdf_states = gpd.read_file("../data/2021_US_States_auto.geojson")
gdf_msas = gpd.read_file("../data/2021_US_CBSA_auto.geojson")

# --- Filter MSAs to only those you have data for
msa_ids_with_data = df_long["metro13"].astype(str).unique()
gdf_msas_data = gdf_msas[gdf_msas["CBSAFP"].astype(str).isin(msa_ids_with_data)]

# --- Clip states: remove only the MSAs that have data
gdf_states_clipped = gdf_states.overlay(gdf_msas_data, how="difference")

# --- Save for later use
gdf_states_clipped.to_file(
    "../data/2021_US_States_clipped_auto.geojson", driver="GeoJSON"
)

In [88]:
with Path.open("../data/2021_US_CBSA_auto.geojson") as f:
    msa_geo = json.load(f)
with Path.open("../data/2021_US_States_clipped_auto.geojson") as f:
    states_geo = json.load(f)

# --- Normalize feature IDs ---
for feat in msa_geo["features"]:
    feat["properties"]["region_id"] = feat["properties"]["CBSAFP"]

for feat in states_geo["features"]:
    feat["properties"]["region_id"] = feat["properties"]["STATEFP"].lstrip("0")

combined_geo = {
    "type": "FeatureCollection",
    "features": msa_geo["features"] + states_geo["features"],
}

In [89]:
output_path = Path("../data/combined_US_regions_auto.geojson")

with output_path.open("w", encoding="utf-8") as f:
    json.dump(combined_geo, f, ensure_ascii=False, indent=2, default=str)

In [ ]:
import altair as alt
import geopandas as gpd

# Load the single combined GeoJSON file
combined_gdf = gpd.read_file("../data/combined_US_regions.geojson")

# Make sure IDs/names exist as columns
combined_gdf["Location Name"] = combined_gdf["NAME"]
combined_gdf["Location ID"] = combined_gdf["region_id"]

# Altair expects JSON geometry, so convert using to_json()
combined_json = json.loads(combined_gdf.to_json())

alt.Chart(alt.Data(values=combined_json["features"])).mark_geoshape(
    stroke="black", strokeWidth=0.75
).encode(
    color=alt.value("lightgrey"),
    tooltip=[
        alt.Tooltip("properties.NAME:N", title="Location Name"),
        alt.Tooltip("properties.region_id:N", title="Location ID"),
    ],
).project(type="albers").properties(width=1000, height=800, title="Combined Map")